## Imports

In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import regex as re
import pandas as pd
import numpy as np
import random 
import pycountry
np.random.seed = 42
random.seed(42)

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)

## Data Format

In [34]:
mailchimp_data = pd.read_csv('./data/MailChimp cleaned records headers.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)
mailchimp_data

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS


The mailchimp dataset comes from the salesforce dataset. This Cleaned dataset would only include emails that have bounced for one reason or another. It will not include any members who have opted out of the email service or those that are receiving emails without any complications. It is important to note that updating information within salesforce for those who have opted out may re-instantiate the email services so we are specifically working with cleaned/bounced records. Additionally, the datasets created for those who are Subscribed and Unsubscribed will have different column names than the Cleaned dataset here (ie Clean_Time, Clean_campaign_title, ect). Therefore any function created here may not work for those datasets. 

In [35]:
saa_pride_data = pd.read_excel('./data/SAA Pride member reports headings.xlsx')
saa_pride_data.reset_index(inplace = True)
saa_pride_data

,index,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind


The Stanford Alumni Association has it's own dataset that may or may not have additional or more recent data on some of the members. It may also have outdated data. Students are given an email but when they become alumni the email needs to be updated. Whether it is updated to specifically an 'alumni.stanford.edu' address or to another would be at the students discrepancy and isn't always updated.

## Helper Functions

In [36]:
def removeEmailDomain(email):
    email = str(email)
    #if email string contains @ symbol return the handle
    if email.find('@')>-1:
        return email.split('@')[0]
    else:
        return 'n/a'

In [213]:
def mapCountry(countryStr):
    if countryStr not in [np.nan, None, 'nan','*', '','N/A','n/a']:
        return pycountry.countries.search_fuzzy(countryStr)[0].alpha_3
    return 'n/a'

In [38]:
def ohe(df, column):    
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(sparse=False, handle_unknown="error")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train,
                                     columns=col_names, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
    return df

In [207]:
def parseDegreeCol(df,deg_colName='short_degree_string'):
    vectorizer = CountVectorizer()
    
    t = vectorizer.fit_transform(df[deg_colName])
    
    # Remove original degree column from original dataframe
    df = df.drop(columns=[deg_colName])
    
    # Create new vectorized degree columns and concatenate
    deg_df =  pd.DataFrame(t.toarray(),columns=vectorizer.get_feature_names_out(),index = df.index)
    if 'nan' in deg_df.columns:
        deg_df.drop(columns = 'nan')
    return pd.concat([df,deg_df],axis = 1).fillna(0)

In [199]:
vectorizer = CountVectorizer()

t = vectorizer.fit_transform(df_mc_and_saa_subset['short_degree_string'])
pd.DataFrame(t.toarray(),columns=vectorizer.get_feature_names_out())

,bs,jd,ma,mba,md,ms,phd
0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0
6,0,0,1,0,0,0,1
7,0,0,0,0,0,1,0
8,0,0,0,0,0,0,1
9,0,1,1,0,0,0,0


In [40]:
def cleanDegreeStr(degree):
    no_nums_puncs = re.sub('[;,\'*0-9]', ' ', degree).strip()
    return re.sub('[ ]+', ' ', no_nums_puncs).strip()

### Pokemon Data

In [41]:
df_saa = pd.read_excel('./data/SAA_Pokemon_FakeDB.xlsx')
df_saa

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind
0,NaN,2004.0,Shanghai,NaN,*,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growlithe,Ice,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Madrid,NaN,China,NaN,NaN,weedleg4046@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Weedle,Grass,NaN,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,'82,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Seoul,NaN,Kuwait,NaN,NaN,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,NaN,NaN,Aerodactyl,Electric,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,London,NaN,NaN,NaN,*,*,NaN,NaN,Japan,NaN,NaN,NaN,Pinsir,Fire,NaN,pinsirfire4582@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"JD '94, PhD '97",NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,London,NaN,USA,NaN,775 0678-214,*,NaN,NaN,NaN,NaN,NaN,NaN,Horsea,Ice,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,NaN,NaN,Boston,MA,Japan,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Smeargle,Electric,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '96, MBA '87",NaN,NaN,NaN,NaN,NaN,NaN
3996,NaN,2000.0,Shanghai,NaN,NaN,NaN,*,kabutops.steel1285@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Kabutops,Steel,NaN,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BS '93,NaN,NaN,NaN,NaN,NaN,NaN
3997,NaN,NaN,NaN,NaN,Kuwait,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Slowking,Dragon,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '84, JD '93",NaN,NaN,NaN,NaN,NaN,NaN
3998,NaN,NaN,NaN,NaN,Japan,NaN,NaN,larvitar.electric9778@stanfordalumni.org,NaN,NaN,United States,NaN,NaN,NaN,Larvitar,Electric,NaN,l.electric7920@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MBA '92, '98, JD '86",NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess SAA dataframe

## Filter Columns

In [42]:
# Filter necessary columns
df_saa = df_saa.filter(['first_name', 'last_name',
                        'home_country', 'home_email_address', 'short_degree_string',
                        'bus_email_address', 'bus_country', 'email_switch',
                        'saa_email_address', 'gsb_email_address', 'other_email_address'])
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,Growlithe,Ice,*,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN
1,Weedle,Grass,China,weedleg4046@stanfordalumni.org,'82,NaN,China,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN
2,Aerodactyl,Electric,Kuwait,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,*,NaN,NaN
3,Pinsir,Fire,NaN,*,"JD '94, PhD '97",NaN,Japan,pinsirfire4582@gmail.com,*,NaN,NaN
4,Horsea,Ice,USA,*,NaN,NaN,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3995,Smeargle,Electric,Japan,*,"PhD '96, MBA '87",NaN,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN
3996,Kabutops,Steel,NaN,kabutops.steel1285@stanfordalumni.org,BS '93,NaN,China,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN
3997,Slowking,Dragon,Kuwait,*,"PhD '84, JD '93",NaN,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN
3998,Larvitar,Electric,Japan,larvitar.electric9778@stanfordalumni.org,"MBA '92, '98, JD '86",NaN,United States,l.electric7920@stanfordalumni.org,NaN,NaN,NaN


## Replace nulls

In [43]:
df_saa.fillna('n/a',inplace=True)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,Growlithe,Ice,*,n/a,n/a,n/a,n/a,n/a,*,n/a,n/a
1,Weedle,Grass,China,weedleg4046@stanfordalumni.org,'82,n/a,China,n/a,w.grass5053@alumni.stanford.edu,n/a,n/a
2,Aerodactyl,Electric,Kuwait,aerodactyl.electric2974@alumni.stanford.edu,n/a,n/a,Kuwait,n/a,*,n/a,n/a
3,Pinsir,Fire,n/a,*,"JD '94, PhD '97",n/a,Japan,pinsirfire4582@gmail.com,*,n/a,n/a
4,Horsea,Ice,USA,*,n/a,n/a,n/a,hice7313@stanfordalumni.org,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,Smeargle,Electric,Japan,*,"PhD '96, MBA '87",n/a,n/a,smeargleelectric9444@gmail.com,*,n/a,n/a
3996,Kabutops,Steel,n/a,kabutops.steel1285@stanfordalumni.org,BS '93,n/a,China,k.steel5317@alumni.stanford.edu,n/a,n/a,n/a
3997,Slowking,Dragon,Kuwait,*,"PhD '84, JD '93",n/a,n/a,n/a,slowkingd5563@stanfordalumni.org,n/a,n/a
3998,Larvitar,Electric,Japan,larvitar.electric9778@stanfordalumni.org,"MBA '92, '98, JD '86",n/a,United States,l.electric7920@stanfordalumni.org,n/a,n/a,n/a


## Strip and lower case

In [44]:
for col in df_saa.columns:    
    df_saa[col] = df_saa[col].astype(str).str.lower().str.strip()
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,*,n/a,n/a
1,weedle,grass,china,weedleg4046@stanfordalumni.org,'82,n/a,china,n/a,w.grass5053@alumni.stanford.edu,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974@alumni.stanford.edu,n/a,n/a,kuwait,n/a,*,n/a,n/a
3,pinsir,fire,n/a,*,"jd '94, phd '97",n/a,japan,pinsirfire4582@gmail.com,*,n/a,n/a
4,horsea,ice,usa,*,n/a,n/a,n/a,hice7313@stanfordalumni.org,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,*,"phd '96, mba '87",n/a,n/a,smeargleelectric9444@gmail.com,*,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285@stanfordalumni.org,bs '93,n/a,china,k.steel5317@alumni.stanford.edu,n/a,n/a,n/a
3997,slowking,dragon,kuwait,*,"phd '84, jd '93",n/a,n/a,n/a,slowkingd5563@stanfordalumni.org,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778@stanfordalumni.org,"mba '92, '98, jd '86",n/a,united states,l.electric7920@stanfordalumni.org,n/a,n/a,n/a


## Remove domains from each email. Retain handles

In [45]:
email_cols = ['home_email_address', 'bus_email_address', 'email_switch',
              'saa_email_address', 'gsb_email_address', 'other_email_address']
for email in email_cols:
    df_saa[email] = df_saa[email].apply(removeEmailDomain)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,'82,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,n/a,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,n/a,n/a,"jd '94, phd '97",n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,n/a,n/a,n/a,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,"phd '96, mba '87",n/a,n/a,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285,bs '93,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,"phd '84, jd '93",n/a,n/a,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,"mba '92, '98, jd '86",n/a,united states,l.electric7920,n/a,n/a,n/a


## Clean Degree string

In [47]:
df_saa.short_degree_string = df_saa.short_degree_string.apply(cleanDegreeStr)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,n/a,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,n/a,n/a,jd phd,n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,n/a,n/a,n/a,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,phd mba,n/a,n/a,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,n/a,kabutops.steel1285,bs,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,phd jd,n/a,n/a,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,mba jd,n/a,united states,l.electric7920,n/a,n/a,n/a


# Mail Chimp Dataframe

In [63]:
df_mailchimp = pd.read_csv('./data/Fake_MailChimp_cleaned_Pokemon.csv')
df_mailchimp

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS
0,slakoth.normal3945@gmail.com,Slakoth,Normal,False,NaN,Texas,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e.rock7454@gmail.com,Espeon,Rock,True,F,DC Area,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rhydonghost7966@alumni.stanford.edu,Rhydon,Ghost,False,M,Bay Area,NaN,USA,MBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,porygong9247@stanfordalumni.org,Porygon,Grass,False,M,Bay Area,NaN,Japan,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tangelagrass1376@gmail.com,Tangela,Grass,False,NaN,New England,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,c.electric7518@gmail.com,Chansey,Steel,True,F,Other US,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,blissey.ghost4154@gmail.com,Blissey,Ghost,False,M,New England,NaN,Macao Special Administrative Region of China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess mailchimp dataframe

## Filter necessary columns

In [64]:
# Filter necessary columns
df_mailchimp = df_mailchimp.filter(['First Name', 'Last Name', 'Email Address',
                                         'Degree', 'Country'])
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,Slakoth,Normal,slakoth.normal3945@gmail.com,NaN,USA
1,Espeon,Rock,e.rock7454@gmail.com,NaN,United States
2,Rhydon,Ghost,rhydonghost7966@alumni.stanford.edu,MBA,USA
3,Porygon,Grass,porygong9247@stanfordalumni.org,MS,Japan
4,Tangela,Grass,tangelagrass1376@gmail.com,NaN,United States
5,Chansey,Steel,c.electric7518@gmail.com,NaN,USA
6,Blissey,Ghost,blissey.ghost4154@gmail.com,NaN,Macao Special Administrative Region of China


## Replace nulls with 'n/a'

In [65]:
df_mailchimp.fillna('n/a',inplace=True)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,Slakoth,Normal,slakoth.normal3945@gmail.com,n/a,USA
1,Espeon,Rock,e.rock7454@gmail.com,n/a,United States
2,Rhydon,Ghost,rhydonghost7966@alumni.stanford.edu,MBA,USA
3,Porygon,Grass,porygong9247@stanfordalumni.org,MS,Japan
4,Tangela,Grass,tangelagrass1376@gmail.com,n/a,United States
5,Chansey,Steel,c.electric7518@gmail.com,n/a,USA
6,Blissey,Ghost,blissey.ghost4154@gmail.com,n/a,Macao Special Administrative Region of China


## Strip and lowercase all names and emails

In [66]:
for col in ['First Name','Last Name','Email Address','Degree']:
    df_mailchimp[col] = df_mailchimp[col].str.lower().str.strip()
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945@gmail.com,n/a,USA
1,espeon,rock,e.rock7454@gmail.com,n/a,United States
2,rhydon,ghost,rhydonghost7966@alumni.stanford.edu,mba,USA
3,porygon,grass,porygong9247@stanfordalumni.org,ms,Japan
4,tangela,grass,tangelagrass1376@gmail.com,n/a,United States
5,chansey,steel,c.electric7518@gmail.com,n/a,USA
6,blissey,ghost,blissey.ghost4154@gmail.com,n/a,Macao Special Administrative Region of China


## Remove email domain

In [67]:
df_mailchimp['Email Address'] = df_mailchimp['Email Address'].apply(removeEmailDomain)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,n/a,USA
1,espeon,rock,e.rock7454,n/a,United States
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,Japan
4,tangela,grass,tangelagrass1376,n/a,United States
5,chansey,steel,c.electric7518,n/a,USA
6,blissey,ghost,blissey.ghost4154,n/a,Macao Special Administrative Region of China


## Convert Country to 3 digit code

In [68]:
# df_mailchimp.Country = df_mailchimp.Country.apply(mapCountry)
# df_mailchimp

## Clean Degree String

In [69]:
df_mailchimp.Degree = df_mailchimp.Degree.apply(cleanDegreeStr)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,n/a,USA
1,espeon,rock,e.rock7454,n/a,United States
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,Japan
4,tangela,grass,tangelagrass1376,n/a,United States
5,chansey,steel,c.electric7518,n/a,USA
6,blissey,ghost,blissey.ghost4154,n/a,Macao Special Administrative Region of China


Converting to 3-char country code is time consuming. Reserving that conversion after subsetting

# Find Best Match for each MailChimp record

Finding the best match for each mail chimp record is a 3 step process
- Map the MailChimp record to the Stanford Alumni dataframe
- Subset the entire dataframe by the first name
- Calculate the cosine similarity for the resulting subset

Few edge cases to note:
- Subset by just the first name, in case a user has changed their last name but change their emails or contact info
- We check for similarity of the 'bounced' MailChimp email handle against *all* possible email fields
- We check for similarity of the 'bounced' MailChimp country field against *all* possible country fields

The result returned is a dictionary of 
- Dataframe of data in order of score (key: `df`)
- List of index of records in stanford alumni dataframe in order of score (key: `idx`)
- List of cosin similarity scores (key: `score`)

In [171]:
def map_MailChimpData_to_SAA_DF(mc_rec_idx,df_mc=df_mailchimp):
    mc_rec = df_mc.iloc[mc_rec_idx]
    target_dict = {'first_name': mc_rec['First Name'], 
                   'last_name': mc_rec['Last Name'],
                   
                   'home_email_address': mc_rec['Email Address'],
                   'bus_email_address': mc_rec['Email Address'],
                   'email_switch': mc_rec['Email Address'],
                   'saa_email_address': mc_rec['Email Address'],
                   'gsb_email_address': mc_rec['Email Address'],
                   'other_email_address': mc_rec['Email Address'],
                   
                    'home_country': mc_rec['Country'],                   
                    'bus_country': mc_rec['Country'],          
                   
                    'short_degree_string': mc_rec['Degree']
                  
                  }
    df =  pd.DataFrame(target_dict,index=['mc_'+str(mc_rec_idx)])
    return df

In [172]:
map_MailChimpData_to_SAA_DF(0).iloc[0]

first_name                        slakoth
last_name                          normal
home_email_address     slakoth.normal3945
bus_email_address      slakoth.normal3945
email_switch           slakoth.normal3945
saa_email_address      slakoth.normal3945
gsb_email_address      slakoth.normal3945
other_email_address    slakoth.normal3945
home_country                          USA
bus_country                           USA
short_degree_string                   n/a
Name: mc_0, dtype: object

In [156]:
print(df_mailchimp.shape)

(7, 5)


In [216]:
results_dict = {}
for i in range(0,df_mailchimp.shape[0]):
    #map the data of a mailchimp record to the format in SAA df
    df_mapped_mc_rec = map_MailChimpData_to_SAA_DF(i)
    
    #subsetting the original SAA df by the first name
    df_saa_subset = df_saa[df_saa['first_name'] == df_mapped_mc_rec.iloc[0]['first_name']]
    
    #Concattenating the 2 dataframes together
    df_mc_and_saa_subset = pd.concat([df_mapped_mc_rec,df_saa_subset], axis = 0)
    
    #Changing degree column to vectorized columns
    df_mc_and_saa_subset = parseDegreeCol(df=df_mc_and_saa_subset,deg_colName='short_degree_string')
    
    #Map the country to their 3 digit codes
    df_mc_and_saa_subset.home_country = df_mc_and_saa_subset.home_country.apply(mapCountry)
    df_mc_and_saa_subset.bus_country = df_mc_and_saa_subset.bus_country.apply(mapCountry)
    
    
    ohe_df = ohe(df_mc_and_saa_subset, df_mc_and_saa_subset.columns)
    ohe_df.drop(columns = df_mc_and_saa_subset.columns, inplace = True)
    
ohe_df

C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Func

C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Func

C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Func

C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Func

,first_name_x0_blissey,last_name_x0_dark,last_name_x0_dragon,last_name_x0_fairy,last_name_x0_fire,last_name_x0_ghost,last_name_x0_ice,last_name_x0_poison,last_name_x0_rock,last_name_x0_steel,last_name_x0_water,home_email_address_x0_bfairy6293,home_email_address_x0_bice5800,home_email_address_x0_blissey.ghost4154,home_email_address_x0_blisseyd5981,home_email_address_x0_blisseyf465,home_email_address_x0_n/a,bus_email_address_x0_blissey.ghost4154,bus_email_address_x0_blissey.rock4438,bus_email_address_x0_blisseywater628,bus_email_address_x0_n/a,email_switch_x0_b.water9408,email_switch_x0_bice9725,email_switch_x0_blissey.ghost4154,email_switch_x0_blisseyf5884,email_switch_x0_blisseyi5337,email_switch_x0_blisseyr2075,email_switch_x0_blisseyr3225,email_switch_x0_blisseys3334,email_switch_x0_n/a,saa_email_address_x0_bdragon9208,saa_email_address_x0_bice7796,saa_email_address_x0_blissey.ghost4154,saa_email_address_x0_blissey.ice4795,saa_email_address_x0_bwater5537,saa_email_address_x0_n/a,gsb_email_address_x0_b.ice9706,gsb_email_address_x0_blissey.ghost4154,gsb_email_address_x0_n/a,other_email_address_x0_blissey.ghost4154,other_email_address_x0_n/a,home_country_x0_CHN,home_country_x0_JPN,home_country_x0_KWT,home_country_x0_MAC,home_country_x0_USA,home_country_x0_n/a,bus_country_x0_CHN,bus_country_x0_JPN,bus_country_x0_KWT,bus_country_x0_MAC,bus_country_x0_USA,bus_country_x0_n/a,bs_x0_0,bs_x0_1,jd_x0_0,jd_x0_1,ma_x0_0,ma_x0_1,mba_x0_0,mba_x0_1,md_x0_0,md_x0_1,ms_x0_0,ms_x0_1,phd_x0_0,phd_x0_1
mc_6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
390,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
600,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
754,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
802,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
850,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
858,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
977,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1430,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1545,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
results_dict = {}
for i in range(0,df_mailchimp.shape[1]+1):
#     target = mailchimp_poke2.iloc[i]
#     target_dict = {'first_name': [target[1]], 'last_name': [target[2]],\
#                emails[0]: [target[0]],\
#                emails[1]: [target[0]],\
#                emails[2]: [target[0]],\
#                emails[3]: [target[0]],\
#                emails[4]: [target[0]],\
#                emails[5]: [target[0]],\
#                    handles[0]: [target[4]],\
#                    handles[1]: [target[4]],\
#                    handles[2]: [target[4]],\
#                    handles[3]: [target[4]],\
#                    handles[4]: [target[4]],\
#                    handles[5]: [target[4]],\
#                'home_country': [target[3]]}
#     df = pd.DataFrame.from_dict(target_dict)
#     subset_saa = saa_poke2[saa_poke2['first_name'] == df.loc[0,'first_name']] 
    #the 0 is calling for row, so it does not return a series
#     subset_saa_new = pd.concat([df,subset_saa], axis = 0)
    ohe_df = ohe(subset_saa_new, subset_saa_new.columns)
    ohe_df.drop(columns = subset_saa_new.columns, inplace = True)
    y = np.array(ohe_df.iloc[0])
    y = y.reshape(1,-1)
    cos_sim = cosine_similarity(ohe_df, y)
    cos_sim = pd.DataFrame(data=cos_sim, index=ohe_df.index).sort_values(by=0, ascending=False) #[1:]
    results = list(cos_sim.index)
    results_df = subset_saa_new.loc[results]
    results_dict[i] = results_df

In [ ]:
# saa_poke2.fillna(value='Not Available', inplace=True)
# emails = ['home_email_address', 'bus_email_address', 'email_switch', 'saa_email_address',\
#          'gsb_email_address', 'other_email_address']
# handles = []
# for x in emails:
#     for i in range(0,saa_poke2.shape[0]):
#         if '@' in saa_poke2[x][i]:
#             saa_poke2[x+'_handle'] = saa_poke2[x].str.split('@').str[0]
#         else:
#             saa_poke2[x+'_handle'] = saa_poke2[x]


## Function

In [ ]:
results_dict = {}
for i in range(0,df_mailchimp.shape[1]+1):
    target = mailchimp_poke2.iloc[i]
    target_dict = {'first_name': [target[1]], 'last_name': [target[2]],\
               emails[0]: [target[0]],\
               emails[1]: [target[0]],\
               emails[2]: [target[0]],\
               emails[3]: [target[0]],\
               emails[4]: [target[0]],\
               emails[5]: [target[0]],\
                   handles[0]: [target[4]],\
                   handles[1]: [target[4]],\
                   handles[2]: [target[4]],\
                   handles[3]: [target[4]],\
                   handles[4]: [target[4]],\
                   handles[5]: [target[4]],\
               'home_country': [target[3]]}
    df = pd.DataFrame.from_dict(target_dict)
    subset_saa = saa_poke2[saa_poke2['first_name'] == df.loc[0,'first_name']] 
    #the 0 is calling for row, so it does not return a series
    subset_saa_new = pd.concat([df,subset_saa], axis = 0)
    ohe_df = ohe(subset_saa_new, subset_saa_new.columns)
    ohe_df.drop(columns = subset_saa_new.columns, inplace = True)
    y = np.array(ohe_df.iloc[0])
    y = y.reshape(1,-1)
    cos_sim = cosine_similarity(ohe_df, y)
    cos_sim = pd.DataFrame(data=cos_sim, index=ohe_df.index).sort_values(by=0, ascending=False) #[1:]
    results = list(cos_sim.index)
    results_df = subset_saa_new.loc[results]
    results_dict[i] = results_df

In [ ]:
target[4]

In [ ]:
subset_saa_new

In [ ]:
mailchimp_poke2

In [ ]:
name_first = input('First Name: ')

results_dict[mailchimp_poke2[mailchimp_poke2['First Name']
                             == name_first].index[0]].head(5)

These are the 5 most likely matches.

## Next Steps

Next step would be to take a proactive approach to mitigate the number of emails that bounce in the future. We would suggest using the salesforce dataset to check for recent grads to reach out before they lose their student emails and ask for updated contact information and their subsequent plans after graduation. It would be easier to update records proactively when we still have accurate contact information. For subsequent plans after graduation, this would be to keeping location information of members to more accurately send regional events and functions. 